## Predict

이미 훈련된 RetinaNet 모델을 로드하여 객체 검출을 하는 과정입니다. Train시와 동일한 환경을 구축하여야 합니다.

In [ ]:
from __future__ import division

import cv2
import numpy as np
import os
from six.moves import zip
import time

from keras.layers import Input
from keras.optimizers import adam
from keras_retinanet.losses import focal, smooth_l1
from keras_retinanet.models.resnet import ResNet50RetinaNet

### 데이터셋 설정

아래는 데이터셋 및 객체 검출 설정에 필요한 환경 변수입니다. 사용하는 데이터셋 및 모델에 맞춰서 변경하시기 바랍니다.

 - `classes` : 데이터내에 구성하는 모든 클래스의 정보를 담은 dict입니다. 키는 클래스의 일련변호, 값은 각 클래스의 이름으로로 지정해야 합니다. 일련번호는 숫자 0부터 중복과 누락없이 채워져야 합니다. 훈련시에 사용한 `classes` dict를 키-값 반전해서 사용하면 됩니다.
 - `weight_file_path` : 입력 weight 파일의 경로 및 이름을 지정합니다.
 - `predict_source` : 객체 검출을 할 대상 이미지 파일이 있는 디렉토리를 지정합니다.
 - `predict_result` : 객체 검출 결과가 출력될 디렉토리를 지정합니다. 자동으로 해당 디렉토리를 생성하지 않으므로 미리 생성해두셔야 합니다.
 - `bbox_threshold` : 검출된 경계 상자를 양성(positive)로 판정할 임계값입니다.
 - `resize` : 이미지가 지나치게 크면 리사이즈하여 검출할지를 지정하는 부울 값입니다.

In [ ]:
classes = None
weight_file_path = './retinanet.h5'
predict_source = './images'
predict_result = './results'
bbox_threshold = 0.5
resize = True

if classes is None:
    classes = {0: 'aeroplane', 1: 'bicycle', 2: 'bird', 3: 'boat', 4: 'bottle',
               5: 'bus', 6: 'car', 7: 'cat', 8: 'chair', 9: 'cow',
               10: 'diningtable', 11: 'dog', 12: 'horse', 13: 'motorbike', 14: 'person',
               15: 'pottedplant', 16: 'sheep', 17: 'sofa', 18: 'train', 19: 'tvmonitor'}

### 모델 빌드

RetinaNet with ResNet50 모델을 빌드하고 모델의 weight 파일을 로드합니다.

In [ ]:
image_input = Input((None, None, 3))
model = ResNet50RetinaNet(image_input, num_classes=len(classes), weights='imagenet', nms=True)

model.load_weights(weight_file_path)
model.compile(
    loss={'regression': smooth_l1(), 'classification': focal()},
    optimizer=adam(lr=1e-5, clipnorm=0.001)
)

### 검출 시작

로드한 모델을 이용해서 소스 이미지의 객체 검출을 실행하고 출력합니다.

In [ ]:
def resize_image(img, min_side, max_side):
    (rows, cols, _) = img.shape
    smallest_side = min(rows, cols)
    scale = min_side / smallest_side
    largest_side = max(rows, cols)
    if largest_side * scale > max_side:
        scale = max_side / largest_side

    img = cv2.resize(img, None, fx=scale, fy=scale)

    return img, scale

for dir_path, _, filenames in os.walk(predict_source):
    for filename in filenames:
        if not filename.endswith(('.jpg', '.png')):
            continue

        # 이미지 로드
        start = time.time()
        file_path = os.path.join(dir_path, filename)
        print(file_path)
        img = cv2.imread(file_path, cv2.IMREAD_COLOR)

        # 정규화 및 리사이즈
        predict = img.copy()
        cv2.normalize(img, predict, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        scale = 1
        if resize:
            predict, scale = resize_image(predict, 720, 1280)
        predict = np.expand_dims(predict, axis=0)

        # 검출
        _, _, detections = model.predict_on_batch(predict)

        # 결과 후처리
        predicted_labels = np.argmax(detections[0, :, 4:], axis=1)
        scores = np.max(detections[0, :, 4:], axis=1)

        for idx, (label, score) in enumerate(zip(predicted_labels, scores)):
            if score < bbox_threshold:
                continue
            b = detections[0, idx, :4] / scale
            b = b.astype(int)

            print('Label: {}, Score: {}, LTRB of the boundary box: {}, {}, {}, {}'.format(classes[label], score, b[0], b[1], b[2], b[3]))

            cv2.rectangle(img, (b[0], b[1]), (b[2], b[3]), (0, 255, 0), 3)
            caption = "{} {:.3f}".format(classes[label], score)
            cv2.putText(img, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 0, 0), 3)
            cv2.putText(img, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1.5, (255, 255, 255), 2)

        cv2.imwrite(os.path.join(predict_result, filename), img)

        print("Processing time: {}".format(time.time() - start))